## Flight duration model: Pipeline stages

- You're going to create the stages for the flights duration model pipeline. You will use these in the next exercise to build a pipeline and to create a regression model.

- The StringIndexer, OneHotEncoder, VectorAssembler and LinearRegression classes are already imported.

### Instructions
-     Create an indexer to convert the 'org' column into an indexed column called 'org_idx'.
-     Create a one-hot encoder to convert the 'org_idx' and 'dow' columns into dummy variable columns called 'org_dummy' and 'dow_dummy'.
-     Create an assembler which will combine the 'km' column with the two dummy variable columns. The output column should be called 'features'.
-     Create a linear regression object to predict flight duration.

- You might find it useful to revisit the slides from the lessons in the Slides panel next to the IPython Shell.

In [ ]:
# Convert categorical strings to index values
indexer = StringIndexer(inputCol='org', outputCol='org_idx')

# One-hot encode index values
onehot = OneHotEncoder(
    inputCols=['org_idx', 'dow'],
    outputCols=['org_dummy', 'dow_dummy']
)

# Assemble predictors into a single column
assembler = VectorAssembler(inputCols=['km', 'org_dummy', 'dow_dummy'], outputCol='features')

# A linear regression object
regression = LinearRegression(labelCol='duration')

## Flight duration model: Pipeline model

- You're now ready to put those stages together in a pipeline.

- You'll construct the pipeline and then train the pipeline on the training data. This will apply each of the individual stages in the pipeline to the training data in turn. None of the stages will be exposed to the testing data at all: there will be no leakage!

- Once the entire pipeline has been trained it will then be used to make predictions on the testing data.

- The data are available as flights, which has been randomly split into flights_train and flights_test.

### Instructions
-    Import the class for creating a pipeline.
-    Create a pipeline object and specify the indexer, onehot, assembler and regression stages, in this order.
-    Train the pipeline on the training data.
-    Make predictions on the testing data.

In [ ]:
|# Import class for creating a pipeline
from pyspark.ml import Pipeline

# Construct a pipeline
pipeline = Pipeline(stages=[indexer, onehot, assembler, regression])

# Train the pipeline on the training data
pipeline = pipeline.fit(flights_train)

# Make predictions on the testing data
predictions = pipeline.transform(flights_test)

## SMS spam pipeline

- You haven't looked at the SMS data for quite a while. Last time we did the following:

    -    split the text into tokens
    -    removed stop words
    -    applied the hashing trick
    -    converted the data from counts to IDF and
    -    trained a logistic regression model.

- Each of these steps was done independently. This seems like a great application for a pipeline!

- The Pipeline and LogisticRegression classes have already been imported into the session, so you don't need to worry about that!

### Instructions
-    Create an object for splitting text into tokens.
-    Create an object to remove stop words. Rather than explicitly giving the input column name, use the getOutputCol() method on the previous object.
-    Create objects for applying the hashing trick and transforming the data into a TF-IDF. Use the getOutputCol() method again.
-    Create a pipeline which wraps all of the above steps as well as an object to create a Logistic Regression model.

In [ ]:
from pyspark.ml.feature import Tokenizer, StopWordsRemover, HashingTF, IDF

# Break text into tokens at non-word characters
tokenizer = Tokenizer(inputCol='text', outputCol='words')

# Remove stop words
remover = StopWordsRemover(inputCol='words', outputCol='terms')

# Apply the hashing trick and transform to TF-IDF
hasher = HashingTF(inputCol='terms', outputCol="hash")
idf = IDF(inputCol="hash", outputCol="features")

# Create a logistic regression object and add everything to a pipeline
logistic = LogisticRegression()
pipeline = Pipeline(stages=[tokenizer, remover, hasher, idf, logistic])

## Cross validating simple flight duration model

- You've already built a few models for predicting flight duration and evaluated them with a simple train/test split. However, cross-validation provides a much better way to evaluate model performance.

- In this exercise you're going to train a simple model for flight duration using cross-validation. Travel time is usually strongly correlated with distance, so using the km column alone should give a decent model.

- The data have been randomly split into flights_train and flights_test.

- The following classes have already been imported: LinearRegression, RegressionEvaluator, ParamGridBuilder and CrossValidator.

### Instructions
-    Create an empty parameter grid.
-    Create objects for building and evaluating a linear regression model. The model should predict the "duration" field.
-    Create a cross-validator object. Provide values for the estimator, estimatorParamMaps and evaluator arguments. Choose 5-fold cross validation.
-    Train and test the model across multiple folds of the training data.

In [ ]:
# Create an empty parameter grid
params = ParamGridBuilder().build()

# Create objects for building and evaluating a regression model
regression = LinearRegression(labelCol='duration')
evaluator = RegressionEvaluator(labelCol='duration')

# Create a cross validator
cv = CrossValidator(estimator=regression, estimatorParamMaps=params, evaluator=evaluator, numFolds=5)

# Train and test model on multiple folds of the training data
cv = cv.fit(flights_train)

# NOTE: Since cross-valdiation builds multiple models, the fit() method can take a little while to complete.

## Cross validating flight duration model pipeline

- The cross-validated model that you just built was simple, using km alone to predict duration.

- Another important predictor of flight duration is the origin airport. Flights generally take longer to get into the air from busy airports. Let's see if adding this predictor improves the model!

- In this exercise you'll add the org field to the model. However, since org is categorical, there's more work to be done before it can be included: it must first be transformed to an index and then one-hot encoded before being assembled with km and used to build the regression model. We'll wrap these operations up in a pipeline.

- The following objects have already been created:

    -    params — an empty parameter grid
    -    evaluator — a regression evaluator
    -    regression — a LinearRegression object with labelCol='duration'.

- The StringIndexer, OneHotEncoder, VectorAssembler and CrossValidator classes have already been imported.

### Instructions
-    Create a string indexer. Specify the input and output fields as org and org_idx.
-    Create a one-hot encoder. Name the output field org_dummy.
-    Assemble the km and org_dummy fields into a single field called features.
-    Create a pipeline using the following operations: string indexer, one-hot encoder, assembler and linear regression. Use this to create a cross-validator.

In [ ]:
# Create an indexer for the org field
indexer = StringIndexer(inputCol='org', outputCol='org_idx')

# Create an one-hot encoder for the indexed org field
onehot = OneHotEncoder(inputCols=['org_idx'], outputCols=['org_dummy'])

# Assemble the km and one-hot encoded fields
assembler = VectorAssembler(inputCols=['km', 'org_dummy'], outputCol='features')

# Create a pipeline and cross-validator.
pipeline = Pipeline(stages=[indexer, onehot, assembler, regression])
cv = CrossValidator(estimator=pipeline,
          estimatorParamMaps=params,
          evaluator=evaluator)

## Optimizing flights linear regression

- Up until now you've been using the default hyper-parameters when building your models. In this exercise you'll use cross validation to choose an optimal (or close to optimal) set of model hyper-parameters.

- The following have already been created:

    -    regression — a LinearRegression object
    -    pipeline — a pipeline with string indexer, one-hot encoder, vector assembler and linear regression and
    -    evaluator — a RegressionEvaluator object.

### Instructions
-    Create a parameter grid builder.
-    Add grids for with regression.regParam (values 0.01, 0.1, 1.0, and 10.0) and regression.elasticNetParam (values 0.0, 0.5, and 1.0).
-    Build the grid.
-    Create a cross validator, specifying five folds.

In [ ]:
# Create parameter grid
params = ParamGridBuilder()

# Add grids for two parameters
params = params.addGrid(regression.regParam, [0.01, 0.1, 1.0, 10.0]) \
               .addGrid(regression.elasticNetParam, [0.0, 0.5, 1.0])

# Build the parameter grid
params = params.build()
print('Number of models to be tested: ', len(params))

# Create cross-validator
cv = CrossValidator(estimator=pipeline, estimatorParamMaps=params, evaluator=evaluator, numFolds=5)

## Dissecting the best flight duration model

- You just set up a CrossValidator to find good parameters for the linear regression model predicting flight duration.

- The model pipeline has multiple stages (objects of type StringIndexer, OneHotEncoder, VectorAssembler and LinearRegression), which operate in sequence. The stages are available as the stages attribute on the pipeline object. They are represented by a list and the stages are executed in the sequence in which they appear in the list.

- Now you're going to take a closer look at the pipeline, split out the stages and use it to make predictions on the testing data.

- The following objects have already been created:
    -    cv — a trained CrossValidatorModel object and
    -    evaluator — a RegressionEvaluator object.

- The flights data have been randomly split into flights_train and flights_test.

### Instructions
-    Retrieve the best model.
-    Look at the stages in the best model.
-    Isolate the linear regression stage and extract its parameters.
-    Use the best model to generate predictions on the testing data and calculate the RMSE.

In [ ]:
# Get the best model from cross validation
best_model = cv.bestModel

# Look at the stages in the best model
print(best_model.stages)

# Get the parameters for the LinearRegression object in the best model
best_model.stages[3].extractParamMap()

# Generate predictions on testing data using the best model then calculate RMSE
predictions = best_model.transform(flights_test)
print("RMSE =", evaluator.evaluate(predictions))

## SMS spam optimised

- The pipeline you built earlier for the SMS spam model used the default parameters for all of the elements in the pipeline. It's very unlikely that these parameters will give a particularly good model though. In this exercise you're going to run the pipeline for a selection of parameter values. We're going to do this in a systematic way: the values for each of the hyperparameters will be laid out on a grid and then pipeline will systematically run across each point in the grid.

- In this exercise you'll set up a parameter grid which can be used with cross validation to choose a good set of parameters for the SMS spam classifier.

- The following are already defined:
    -    hasher — a HashingTF object and
    -    logistic — a LogisticRegression object.

### Instructions
-    Create a parameter grid builder object.
-    Add grid points for numFeatures and binary parameters to the HashingTF object, giving values 1024, 4096 and 16384, and True and False, respectively.
-    Add grid points for regParam and elasticNetParam parameters to the LogisticRegression object, giving values of 0.01, 0.1, 1.0 and 10.0, and 0.0, 0.5, and 1.0 respectively.
-    Build the parameter grid.

In [ ]:
# Create parameter grid
params = ParamGridBuilder()

# Add grid for hashing trick parameters
params = params.addGrid(hasher.numFeatures, [1024, 4096, 16384]) \
               .addGrid(hasher.binary, [True, False])

# Add grid for logistic regression parameters
params = params.addGrid(logistic.regParam, [0.01, 0.1, 1.0, 10.0]) \
               .addGrid(logistic.elasticNetParam, [0.0, 0.5, 1.0])

# Build parameter grid
params = params.build()

## Delayed flights with Gradient-Boosted Trees

- You've previously built a classifier for flights likely to be delayed using a Decision Tree. In this exercise you'll compare a Decision Tree model to a Gradient-Boosted Trees model.

- The flights data have been randomly split into flights_train and flights_test.

### Instructions
-    Import the classes required to create Decision Tree and Gradient-Boosted Tree classifiers.
-    Create Decision Tree and Gradient-Boosted Tree classifiers. Train on the training data.
-    Create an evaluator and calculate AUC on testing data for both classifiers. Which model performs better?
-    For the Gradient-Boosted Tree classifier print the number of trees and the relative importance of features.

In [ ]:
# Import the classes required
from pyspark.ml.classification import DecisionTreeClassifier, GBTClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator

# Create model objects and train on training data
tree = DecisionTreeClassifier().fit(flights_train)
gbt = GBTClassifier(maxIter=10).fit(flights_train)

# Compare AUC on testing data
evaluator = BinaryClassificationEvaluator()
print(evaluator.evaluate(tree.transform(flights_test)))
print(evaluator.evaluate(gbt.transform(flights_test)))

# Find the number of trees and the relative importance of features
print(gbt.trees)
print(gbt.featureImportances)

## Delayed flights with a Random Forest

- In this exercise you'll bring together cross validation and ensemble methods. You'll be training a Random Forest classifier to predict delayed flights, using cross validation to choose the best values for model parameters.

- You'll find good values for the following parameters:

    -    featureSubsetStrategy — the number of features to consider for splitting at each node and
    -    maxDepth — the maximum number of splits along any branch.

- Unfortunately building this model takes too long, so we won't be running the .fit() method on the pipeline.

- The RandomForestClassifier class has already been imported into the session.

### Instructions
-    Create a random forest classifier object.
-    Create a parameter grid builder object. Add grid points for the featureSubsetStrategy and maxDepth parameters.
-    Create binary classification evaluator.
-    Create a cross-validator object, specifying the estimator, parameter grid and evaluator. Choose 5-fold cross validation.

In [ ]:
# Create a random forest classifier
forest = RandomForestClassifier()

# Create a parameter grid
params = ParamGridBuilder() \
            .addGrid(forest.featureSubsetStrategy, ['all', 'onethird', 'sqrt', 'log2']) \
            .addGrid(forest.maxDepth, [2, 5, 10]) \
            .build()

# Create a binary classification evaluator
evaluator = BinaryClassificationEvaluator()

# Create a cross-validator
cv = CrossValidator(estimator=forest, estimatorParamMaps=params, evaluator=evaluator, numFolds=5)

## Evaluating Random Forest

- In this final exercise you'll be evaluating the results of cross-validation on a Random Forest model.

- The following have already been created:
    -    cv - a cross-validator which has already been fit to the training data
    -    evaluator — a BinaryClassificationEvaluator object and
    -    flights_test — the testing data.

### Instructions
-    Print a list of average AUC metrics across all models in the parameter grid.
-    Display the average AUC for the best model. This will be the largest AUC in the list.
-    Print an explanation of the maxDepth and featureSubsetStrategy parameters for the best model.
-    Display the AUC for the best model predictions on the testing data.

In [ ]:
# Average AUC for each parameter combination in grid
print(cv.avgMetrics)

# Average AUC for the best model
print(max(cv.avgMetrics))

# What's the optimal parameter value for maxDepth?
print(cv.bestModel.explainParam('maxDepth'))
# What's the optimal parameter value for featureSubsetStrategy?
print(cv.bestModel.explainParam('featureSubsetStrategy'))

# AUC for best model on testing data
print(evaluator.evaluate(cv.transform(flights_test)))